# Dec 31 Branching Test

In [ ]:
# Torch modules
import torch 
import torch.nn as nn 
import torch.nn.functional as F


import sys
sys.path.append("../")

# ConvNN Modules
from Conv2d_NN import * 
from Conv2d_NN_spatial import * 
from Conv1d_NN import * 
from Conv1d_NN_spatial import * 
from pixelshuffle import * 

# Data + Training
from data import * 
from train import * 

## I. Model 

In [ ]:
class Branching2D_Layer(nn.module):
    def __init__(self, in_ch, out_ch1, out_ch2, kernel_size):
        super().__init__()
        
        self.kernel_size = kernel_size
        self.in_ch = in_ch
        self.out_ch1 = out_ch1
        self.out_ch2 = out_ch2
        
        self.Conv2d_Branch = nn.Sequential(
            nn.Conv2d(in_ch, out_ch1, kernel_size), 
            nn.ReLU()
        )

        
        self.Conv2d_NN_Branch = nn.Sequential(
            Conv2d_NN(in_ch, out_ch2, K=kernel_size, stride = kernel_size), 
            nn.ReLU()
        )
        
        self.reduce_channels = nn.Conv2d(out_ch1+out_ch2, (out_ch1 + out_ch2) // 2, 1)
        
    
    def forward(self, x):
        x1 = self.Conv2d_Branch(x)
        x2 = self.Conv2d_NN_Branch(x)
        
        # Debugging for shape
        print("x1 shape: ", x1.shape)
        print("x2 shape: ", x2.shape)
        
        ## Calculate expected Output size of x2
        expected_x1_size = x2.size(2)
        print("expected x1 size: ", expected_x1_size)
        
        ## Calculate padding for x1 to match x2's size 
        total_padding = expected_x1_size - x1.size(2) 
        print("total padding: ", total_padding)
        
        left_padding = total_padding //2 
        right_padding = total_padding - left_padding
        print("Right + Left padding: ", left_padding, right_padding)
        
        ## Apply dynamic padding to x2 
        x1 = F.pad(x1, (left_padding, right_padding), 'constant', 0)
        print("new x1 shape: ", x1.shape)
        
        ## Concatenate the outputs along the channel dimension 
        concat = torch.cat([x1, x2], dim=1)
        print("concatenated shape: ", concat.shape)
        
        ## Reduce the number of channels 
        reduce = self.reduce_channels(concat) 
        print("reduced shape: ", reduce.shape)
        
        return reduce
        


In [10]:
### Playground Tests
ex = torch.randn(32, 3, 64, 64)
print(ex.shape)

Conv2dNN = Conv2d_NN(
    in_channels=3, 
    out_channels=16, 
    K=5, 
    stride=5, 
    padding=0, 
    samples='all', 
    magnitude_type='distance')

Conv2d = nn.Conv2d(
    in_channels=3, 
    out_channels=16, 
    kernel_size=5
    )


torch.Size([32, 3, 64, 64])


In [14]:
x1 = Conv2d.forward(ex)
x2 = Conv2dNN.forward(ex)



# Debugging for shape
print("x1 shape: ", x1.shape)
print("x2 shape: ", x2.shape)

## Calculate expected Output size of x2
expected_x1_size = x2.size(2)
print("expected x1 size: ", expected_x1_size)

## Calculate padding for x1 to match x2's size 
total_padding = expected_x1_size - x1.size(2) 
print("total padding: ", total_padding)

left_padding = total_padding //2 
right_padding = total_padding - left_padding
print("Right + Left padding: ", left_padding, right_padding)

## Apply dynamic padding to x2 
x1 = F.pad(x1, (left_padding, right_padding), 'constant', 0)
print("new x1 shape: ", x1.shape)

## Concatenate the outputs along the channel dimension 
concat = torch.cat([x1, x2], dim=1)
print("concatenated shape: ", concat.shape)

## Reduce the number of channels 
reduce = self.reduce_channels(concat) 
print("reduced shape: ", reduce.shape)




x1 shape:  torch.Size([32, 16, 60, 60])
x2 shape:  torch.Size([32, 16, 64, 64])
expected x1 size:  64
total padding:  4
Right + Left padding:  2 2
new x1 shape:  torch.Size([32, 16, 60, 64])


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 60 but got size 64 for tensor number 1 in the list.